In [2]:
!kaggle competitions download -c house-prices-advanced-regression-techniques


  0%|          | 0.00/199k [00:00<?, ?B/s]
100%|##########| 199k/199k [00:00<00:00, 6.42MB/s]


In [5]:
import zipfile
with zipfile.ZipFile("house-prices-advanced-regression-techniques.zip", "r") as zip_ref:
    zip_ref.extractall("house")

In [9]:
import pandas as pd

### 1. Import the data files

In [24]:
train = pd.read_csv("house\\train.csv")
test = pd.read_csv("house\\test.csv")

In [23]:
test.isnull().sum().sum()

7000